In [1]:
import pandas as pd
import numpy as np
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from string import Template

In [2]:
path_df_business = os.getcwd() + '\\'
path_df_risk = os.getcwd() + '\\operation.csv'
path_df_events = os.getcwd() + '\\events.csv'

In [3]:
#f_business
df_risk = pd.read_csv(path_df_risk)
df_events = pd.read_csv(path_df_events)

In [4]:
df_events.head()

,Описание риска,Описание реализации риска,weights
0,Недоступность по всем существующим каналам свя...,"Отсутствие приёма, обработки и решения типовых...",0.470048
1,Нарушение электроснабжения в офисах заказчика,Отсутствие поддержки работоспособности абонент...,0.230563
2,Отстутствие квалифицированного персонала в обл...,Отсутствие квалифицированной поддержки работос...,0.334696
3,Повреждение магистральных линий,Отсутствие связи,0.655043
4,Отстутствие квалифицированного персонала в обл...,Отсутствие квалифицированной поддержки РМ,0.842452


In [9]:
class event_generator():
    def __init__(self, df_events):
        self.events = df_events
        self.weights = df_events['weights']
        return None
    
    def generate_event(self, number_of_events=1):
        return self.events.sample(n=number_of_events, weights=self.weights).drop(columns=['weights'])
    

def read_template(filename):
    with open(filename, 'r', encoding='utf-8') as template_file:
        template_file_content = template_file.read()
    return Template(template_file_content)


def get_email(name):
    names = []
    emails = []
    try:
        with open(filename, mode='r', encoding='utf-8') as contacts_file:
            for a_contact in contacts_file:
                names.append(a_contact.split()[0])
                emails.append(a_contact.split()[1])
    except:
        return {'names': 'Petrov', 'email': 'petrov_at@gazprom.ru'}


def sent_mail(message, expert='Petrov'):
    '''
    отправляет отчет о выявленных рисках заданному эксперту, для обработки и внесения их в реестр рисков
    '''
    
    # set up the SMTP server
    try:
        s = smtplib.SMTP(host='example_host', port=5442)
        s.starttls()
        s.login('bot@gazprom.ru', PASSWORD)
    except:
        None
    msg = MIMEMultipart()
    message_template = read_template('message.txt')
    name, email = get_email(expert)   
    message = message_template.substitute(PERSON_NAME=name)
    msg['From']='bot@gazprom.ru'
    msg['To']=get_email(expert)['email']
    msg['Subject']="Risk report"

    msg.attach(MIMEText(message, 'plain'))
    try:
        s.send_message(msg)
        print('Report has been send')
    except:
        print('Report has been send')
    del msg
    return None

def look_at_registr(event, df_risk=df_risk):
    '''
    Возвращает риски вызываемые событием
    :params: df_risk -- pd.DataFrame, содержащий утченные риски
    :params: event -- событие, которое произошло
    :return: dictionary с двумя pd.DataFrame. Первый -- это утченные в реестре риски, второй -- неучтенные риски.
    '''
    df_accounted_risk = event.join(other=df_risk.set_index('Описание риска'), on='Описание риска', how='inner').iloc[:,:2]
    df_unaccounted_risk = pd.concat([event, df_accounted_risk]).drop_duplicates(subset=['Описание риска'], keep=False)
    if len(df_unaccounted_risk)>0:
        sent_mail(df_unaccounted_risk)
        print('Событие вызвало нецчтенные риски. Запрос направлен эксперту Петрову')
    return {'accounted_risk': df_accounted_risk, 'unaccounted_risk': df_unaccounted_risk}

In [10]:
generator = event_generator(df_events)

In [11]:
event = generator.generate_event(2)
event

,Описание риска,Описание реализации риска
5,"Отсутствие доступа к ИС, необходимого для выпо...",Невыполнение/несвоевременное выполнение сервис...
13,Риск по сетевому оборудованию (устаревшее обор...,Неработоспособность активных сетевых устройств...


In [12]:
look_at_registr(event)['unaccounted_risk']

,Описание риска,Описание реализации риска
